In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

### Training Data

In [2]:
datadir = 'C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Training'
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
train_images = []
train_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(200,200) )
        train_images.append(img_array)
        train_labels.append(categories.index(category))

In [3]:
len(train_images),len(train_labels)

(2870, 2870)

In [4]:
for i in range(0,len(train_images)):
    train_images[i] = train_images[i].reshape(200*200,)

In [5]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_images.shape, train_labels.shape

((2870, 40000), (2870,))

In [6]:
train_images = train_images/255.0

In [7]:
train_labels

array([0, 0, 0, ..., 3, 3, 3])

### Testing Data

In [8]:
datadir = os.path.normpath('C:/Users/LightSpeed/Light Speed/Deep Learning/Brain Tumor Classification MRI/Testing')
categories = ['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
test_images = []
test_labels = []
for category in categories:
    path = os.path.join(datadir,category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(200,200) )
        test_images.append(img_array)
        test_labels.append(categories.index(category))

In [9]:
len(test_images),len(test_labels)

(394, 394)

In [10]:
for i in range(0,len(test_images)):
    test_images[i] = test_images[i].reshape(200*200,)

In [11]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images.shape, test_labels.shape

((394, 40000), (394,))

In [12]:
test_images = test_images/255.0

In [13]:
import tensorflow as tf
from tensorflow import keras
from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers

In [31]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units=hp.Int('units_'+str(i),
                                           min_value=32,
                                           max_value=4000,
                                           step=100),
                              activation='relu'))
        model.add(layers.Dense(1,activation='softmax'))
        model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
                     loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

SyntaxError: keyword can't be an expression (<ipython-input-31-b0766904f9cd>, line 11)

In [15]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/'),
    project_name='Hypx')

In [16]:
tuner.search(train_images, train_labels,epochs=5)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_layers        |7                 |?                 
units_0           |3932              |?                 
learning_rate     |0.0001            |?                 
units_1           |132               |?                 

Epoch 1/5
90/90 [==============================] - 23s 261ms/step - loss: nan - accuracy: 0.2882
Epoch 2/5
90/90 [==============================] - 6s 67ms/step - loss: nan - accuracy: 0.2878: 0s - loss: nan - accurac
Epoch 3/5
90/90 [==============================] - 6s 65ms/step - loss: nan - accuracy: 0.2878
Epoch 4/5
90/90 [==============================] - 6s 65ms/step - loss: nan - accuracy: 0.2878
Epoch 5/5
90/90 [==============================] - 6s 65ms/step - loss: nan - accuracy: 0.2878
Epoch 1/5


ResourceExhaustedError: in user code:

    C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return f

In [29]:
from sklearn.decomposition import PCA

pca = PCA(n_components=1521)
pca.fit_transform(train_images).shape

(2870, 1521)

In [40]:
np.sqrt(1521)

39.0

In [30]:
train_images_pca = pca.fit_transform(train_images)

In [34]:
tuner_pca = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/'),
    project_name='Hypx_pca')

In [35]:
tuner_pca.search(train_images_pca, train_labels,epochs=5)

Trial 5 Complete [00h 00m 19s]
accuracy: 0.2881533106168111

Best accuracy So Far: 0.28861788908640545
Total elapsed time: 00h 01m 53s
INFO:tensorflow:Oracle triggered exit


In [60]:
train_images_cnn = train_images_pca.reshape(2870,39,39,1)

In [61]:
train_images_cnn.shape

(2870, 39, 39, 1)

In [62]:
def cnn_build_model(hp):
    model = keras.Sequential()
    
    model.add(keras.layers.Conv2D( filters = hp.Int('conv_1_filter',min_value=32, max_value=128, step=1),
        kernel_size =  hp.Int('conv_1_kernel',min_value=3, max_value=30,step=1),
        activation='relu',
        input_shape = (39,39,1)))
    
    for i in range(hp.Int('num_layers',2,20)):
        model.add(keras.layers.Conv2D( filters = hp.Int('conv_filter'+str(i),min_value=32, max_value=128, step=1),
        kernel_size =  hp.Int('conv_kernel'+str(i),min_value=3, max_value=30,step=1),
        activation='relu'
        ))
        
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dens_1_units', min_value=32, max_value=128, step=16),
        activation='relu') ,
    keras.layers.Dense(4,activation='softmax')
    
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                  loss = 'sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

In [64]:
tuner_cnn = RandomSearch(
    cnn_build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory=os.path.normpath('C:/'),
    project_name='Hyp_cnn_x')

In [65]:
tuner_cnn.search(train_images_cnn, train_labels,epochs=5)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |50                |?                 
conv_1_kernel     |6                 |?                 
num_layers        |10                |?                 
conv_filter0      |57                |?                 
conv_kernel0      |20                |?                 
conv_filter1      |80                |?                 
conv_kernel1      |25                |?                 
dens_1_units      |112               |?                 
learning_rate     |0.001             |?                 



Traceback (most recent call last):
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\ops.py", line 1812, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 25 from 15 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv2d_1/Relu, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,15,15,57], [25,25,57,80].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-62-3b42e5ad8aee>", line 12, in cnn_build_model
    activation='relu'
  File "C:\Users\LightS

Invalid model 0/5


Traceback (most recent call last):
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\ops.py", line 1812, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 25 from 15 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv2d_1/Relu, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,15,15,57], [25,25,57,80].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-62-3b42e5ad8aee>", line 12, in cnn_build_model
    activation='relu'
  File "C:\Users\LightS

Invalid model 1/5


...

Invalid model 3/5


Traceback (most recent call last):
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\ops.py", line 1812, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 25 from 15 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv2d_1/Relu, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,15,15,57], [25,25,57,80].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-62-3b42e5ad8aee>", line 12, in cnn_build_model
    activation='relu'
  File "C:\Users\LightS

Invalid model 4/5
Invalid model 5/5


Traceback (most recent call last):
  File "C:\Users\LightSpeed\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\ops.py", line 1812, in _create_c_op
    c_op = pywrap_tf_session.TF_FinishOperation(op_desc)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Negative dimension size caused by subtracting 25 from 15 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv2d_1/Relu, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,15,15,57], [25,25,57,80].

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LightSpeed\anaconda3\envs\gpu\lib\site-packages\kerastuner\engine\hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-62-3b42e5ad8aee>", line 12, in cnn_build_model
    activation='relu'
  File "C:\Users\LightS

RuntimeError: Too many failed attempts to build model.